# Abrir fuente de datos

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [11]:
#Elimina folder
#!rm -rf LC08
!rm -rf spectrograms224_00
#!rm -rf spectrograms224_00/TC
#!rm -rf sismograms224_00
#!rm -rf sismograms224_00/TC

In [ ]:
# Descomprimirlo
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00.zip' &> /dev/null
#!unzip '/content/gdrive/My Drive/Sismogramas/Escenario2/00/sismograms224_00.zip' &> /dev/null
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 +da_rotacion_25_45.zip' &> /dev/null
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 +daRotacion/spectrograms224_00-01I.zip' &> /dev/null
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 210708.zip' &> /dev/null

# Instalando librerias necesarias

In [2]:
#TorchMetrics
!pip install torchmetrics &> /dev/null

## Parámetros

In [3]:
# Define parámetros
class TL:   # Modelos pre-entrenados
  AlexNet   ="AlexNet"
  GoogLeNet ="GoogLeNet"
  ResNet    ="ResNet152"
  VGG       ="VGG19bn"
class FLAGS:
  tls = [TL.AlexNet]          # Modelo TL a considerar en proceso
  carpeta       = 'spectrograms224_00'      # Carpeta de espectrogramas
  classes       = ['HY','LP','TC','TR','VT']  # Lista de clases
  num_classes   = 5           # Número de clases
  batch_size    = 8           # Genera el dataset torch de los espectrogramas
  num_workers   = 2           # Procesos
  learning_rate = 5e-5        # Tasa de aprendizaje
  num_epochs    = 20          # Número de épocas de entrenamiento
  iPorcentajeTiempoInicio = 5           # Porcentaje de tiempo de señal para rotación inicio. Rango 0-100.
  iPorcentajeTiempoFinal  = 6           # Porcentaje de tiempo de señal para rotación fin. Rango 0-100.
  iPorcentajeTiempoInc    = 5           # Incremento
  iPorcentajeTiempoLong   = 20          # Longitud de rangos porcentuales.

### Red Neuronal

In [4]:
# https://analyticsindiamag.com/implementing-alexnet-using-pytorch-as-a-transfer-learning-model-in-multi-class-classification/
# https://towardsdatascience.com/transfer-learning-in-speech-emotion-recognition-d55b6616ba83
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import time
import csv
import itertools
from torchmetrics.functional import accuracy, f1_score, recall, precision, confusion_matrix, cohen_kappa
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Establezca nuestra semilla y otras configuraciones para la reproducibilidad
np.random.seed(42)                        # Establece la semilla para generar números aleatorio numpy
torch.manual_seed(42)                     # Establece la semilla para generar números aleatorio pytorch
torch.cuda.manual_seed(42)                # Establece la semilla para generar números aleatorio pytorch cuda
torch.backends.cudnn.benchmark = False    # True, hace que cuDNN compare múltiples algoritmos de convolución y seleccione el más rápido
torch.backends.cudnn.deterministic = True # True, hace que cuDNN solo use algoritmos de convolución deterministas

# Transformaciones para la imagen
data_transform = transforms.Compose([
  transforms.ToTensor(),
  #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# CreaDataLoaders
def creaDataLoaders(sNombreCarpeta:str, iBatchSize:int, iWorkers:int, lClases):
  # Leemos todos los datos
  all_dataset = datasets.ImageFolder(root=sNombreCarpeta, transform=data_transform, )

  # Cantidades de train y test 80/20
  iTrain=int(len(all_dataset)*0.8)
  iTest=len(all_dataset)-iTrain

  # Dividiendo dataset train y test con elementos aleatorios
  train_dataset, test_dataset = torch.utils.data.random_split(all_dataset, [iTrain, iTest], generator=torch.Generator().manual_seed(42))

  #test_dataset = datasets.ImageFolder(root='spectrograms/'+ESTACION+'/'+CANAL+'/test', transform=data_transform)
  trainloader = DataLoader(train_dataset, batch_size=iBatchSize, shuffle=True,  num_workers=iWorkers)
  testloader  = DataLoader(test_dataset,  batch_size=iBatchSize, shuffle=False, num_workers=iWorkers)

  # (OPCIONAL)Creando un dataloader total de todos los datos, para algun caso, generalmente TEST
  all_dataloader  = DataLoader(all_dataset, batch_size=iBatchSize, shuffle=False, num_workers=iWorkers)

  # Estadisticas
  print('Clases: {} Datos de Entrenamiento:{} Prueba:{} Total:{}'.format(lClases, len(train_dataset), len(test_dataset),len(all_dataset)))

  # Retornando Datasets, Loaders y Longitudes
  return all_dataloader, trainloader, testloader, len(all_dataset), len(train_dataset), len(test_dataset)

# Modelo
def creaModelo(sModeloTL:str):
  #Modelos pre-entrenados de PyTorch ===========================================
  if sModeloTL==TL.AlexNet:
    #tModelo=torchvision.models.alexnet(pretrained=True)
    tModelo = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)     # https://pytorch.org/hub/pytorch_vision_alexnet
  elif sModeloTL==TL.GoogLeNet:
    #tModelo = torchvision.models.googlenet(pretrained=True)
    tModelo = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)   # https://pytorch.org/hub/pytorch_vision_googlenet
  elif sModeloTL==TL.VGG:
    #tModelo = torchvision.models.vgg19_bn(pretrained=True)
    tModelo = torch.hub.load('pytorch/vision:v0.10.0', 'vgg19_bn', pretrained=True)    # https://pytorch.org/hub/pytorch_vision_vgg
  elif sModeloTL==TL.ResNet:
    #tModelo = torchvision.models.resnet152(pretrained=True)
    tModelo = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)   # https://pytorch.org/hub/pytorch_vision_resnet
  else:
    print("Error modelo indefinido")

  #Descripción modelo
  #tModelo.eval()

  #Actualizando configuración de modelo ========================================
  #Actualizando el segundo clasifidor de tal forma que se tenga 5 clases de salida
  if sModeloTL==TL.AlexNet:
    tModelo.classifier[4] = nn.Linear(4096, 1024)               #Actualizando el segundo clasificador
    tModelo.classifier[6] = nn.Linear(1024, FLAGS.num_classes)  #Actualización del tercer(último) clasificador
  elif sModeloTL==TL.GoogLeNet:
    tModelo.fc = nn.Linear(1024, FLAGS.num_classes)             #Actualizando el último clasificador
  elif sModeloTL==TL.VGG:
    tModelo.classifier[3] = nn.Linear(4096, 1024)               #Actualizando el segundo clasificador
    tModelo.classifier[6] = nn.Linear(1024, FLAGS.num_classes)  #Actualización del tercer(último) clasificador
  elif sModeloTL==TL.ResNet:
    tModelo.fc = nn.Linear(2048, FLAGS.num_classes)             #Actualizando el último clasificador
  else:
    print("Error modelo indefinido")

  #Descripción modelo
  #TL_Modelo.eval()
  return tModelo

# Entrenamiento y prueba =======================================================
def train_model(model, loss_function, optimizer, data_loader):
  # Modelo en modo entrenamiento
  model.train()
  current_loss = 0.0
  current_acc = 0
  # Iterar sobre los datos
  for i, (inputs, labels) in enumerate(data_loader):
    # Enviar input/labels a la GPU
    inputs, labels = inputs.to(device), labels.to(device)
    # Parámetros gradiente a cero
    optimizer.zero_grad()
    with torch.set_grad_enabled(True):
      # Forward (Predecir clases a partir de imágenes)
      outputs = model(inputs)
      _, predictions = torch.max(outputs, 1)
      # Calcule la pérdida según las predicciones y las etiquetas reales
      loss = loss_function(outputs, labels)
      # Backward (Retropropagar la pérdida)
      loss.backward()
      # Ajustar parámetros de acuerdo a los gradientes calculados
      optimizer.step()
    # Estadísticas
    current_loss += loss.item() * inputs.size(0)
    current_acc += torch.sum(predictions == labels.data)
  total_loss = current_loss / len(data_loader.dataset)
  total_acc = current_acc.double() / len(data_loader.dataset)
  return total_loss, total_acc.cpu()

def test_model(model, loss_function, data_loader):
  # Modelo en modo evaluación
  model.eval()
  current_loss = 0.0
  current_acc = 0
  # Iterar sobre los datos
  for i, (inputs, labels) in enumerate(data_loader):
    # Enviar input/labels a la GPU
    inputs, labels = inputs.to(device), labels.to(device)
    # Forward (Predecir clases a partir de imágenes)
    with torch.set_grad_enabled(False):
      outputs = model(inputs)
      _, predictions = torch.max(outputs, 1)
      loss = loss_function(outputs, labels)
    # Estadísticas
    current_loss += loss.item() * inputs.size(0)
    current_acc += torch.sum(predictions == labels.data)
  total_loss = current_loss / len(data_loader.dataset)
  total_acc = current_acc.double() / len(data_loader.dataset)
  return total_loss, total_acc.cpu()

# Guarda en archivo gráficos de Loss y Acurracy de Entrenamiento y Test ========
def plotLossAcurracy(sNombreArchivo:str, vEpocas, vLossTrain, vLossTest, vAccurracyTrain, vAccuracyTest):
  fig = plt.figure(figsize=(12, 4))
  plt.subplot(1,2,1)
  plt.plot(vEpocas, vLossTrain, 'g', label='Training loss')
  plt.plot(vEpocas, vLossTest, 'b', label='Test loss')
  plt.title('Training and Test loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.subplot(1,2,2)
  plt.plot(vEpocas, vAccurracyTrain, 'g', label='Training accuracy')
  plt.plot(vEpocas, vAccuracyTest, 'b', label='Test accuracy')
  plt.title('Training and Test accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()
  #plt.savefig(sNombreArchivo, dpi=100, aspect='normal', bbox_inches='tight', pad_inches=0)
  plt.savefig(sNombreArchivo, dpi=100, bbox_inches='tight', pad_inches=0)
  plt.clf()
  plt.close('all')

# Guarda CVS con metricas y retorna matriz de confusión ========================
def csvMetricas(sNombreArchivo:str, tModelo, tDataLoader, lClases, iEpocas, iDatasetTotal, iDatasetTrain, iDatasetTest):
  # Construyendo arreglo de predicciones y datos reales de DATASET TEST
  vPrediccion = torch.tensor([],dtype=torch.int64).cuda()
  vReal       = torch.tensor([],dtype=torch.int64).cuda()
  with torch.no_grad():
    for i, (inputs, classes) in enumerate(tDataLoader):
      inputs,classes = inputs.to(device), classes.to(device)
      outputs = tModelo(inputs)
      _, preds = torch.max(outputs, 1)
      #print('Clase:{} Predicción:{}'.format(classes, preds))
      vPrediccion = torch.cat((vPrediccion, preds), dim=0)
      vReal       = torch.cat((vReal, classes), dim=0)

  # Generando y guardando en métricas en CSV
  confusionmatrix=confusion_matrix(vPrediccion, vReal, num_classes=len(lClases), task='multiclass').cpu()
  lAccuracyClass=[]
  for i in range(len(lClases)):
    lAccuracyClass.append(100*(confusionmatrix.diag()/confusionmatrix.sum(1))[i].cpu().numpy())
  average_accuracy=((100.0*confusionmatrix.diag()/confusionmatrix.sum(1)).sum())/len(lClases)
  # Creando CSV
  with open(sNombreArchivo, mode='w') as csv_file:
    csv_wrt = csv.writer(csv_file, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_wrt.writerow(['Epochs','Classes','Total','Train','Test','Time','Accuracy','F1','Recall','Precision','Cohen Kappa','Average Acurracy','Class Accuracy'])
    csv_wrt.writerow([iEpocas,'-'.join(lClases), iDatasetTotal, iDatasetTrain, iDatasetTest, tTiempoEntrenamiento,
                      100*accuracy(vPrediccion,     vReal, num_classes=len(lClases), average='macro', task='multiclass').cpu().numpy(),
                      100*f1_score(vPrediccion,     vReal, num_classes=len(lClases), average='macro', task='multiclass').cpu().numpy(),
                      100*recall(vPrediccion,       vReal, num_classes=len(lClases), average='macro', task='multiclass').cpu().numpy(),
                      100*precision(vPrediccion,    vReal, num_classes=len(lClases), average='macro', task='multiclass').cpu().numpy(),
                      100*cohen_kappa(vPrediccion,  vReal, num_classes=len(lClases), task='multiclass').cpu().numpy(),
                      average_accuracy.cpu().numpy()
                    ] + lAccuracyClass
                    )
  # Retorna matriz de confusion calculada
  return confusionmatrix

# Graficar matriz de consusión =================================================
# Calcula matriz de confusión
def plotConfusionMatrixSimple(vConfusionMatrix, lClases, bNormalized:bool=False, sTitle:str='Confusion matrix', cmap=plt.cm.Blues):
  if bNormalized:
    vConfusionMatrix = vConfusionMatrix/vConfusionMatrix.sum(axis=1)[:, np.newaxis]
    #print("Matriz de confusión Normalizeda")
  #else:
    #print('Matriz de Confusión, sin normalización')

  #print(vConfusionMatrix)
  plt.imshow(vConfusionMatrix, interpolation='nearest', cmap=cmap)
  plt.title(sTitle)
  plt.colorbar()
  tick_marks = np.arange(len(lClases))
  plt.xticks(tick_marks, lClases, rotation=0)
  plt.yticks(tick_marks, lClases)

  fmt = '.2f' if bNormalized else '.0f'
  thresh = vConfusionMatrix.max()/2.
  for i, j in itertools.product(range(vConfusionMatrix.shape[0]), range(vConfusionMatrix.shape[1])):
    plt.text(j, i, format(vConfusionMatrix[i,j].item(), fmt), horizontalalignment="center", color="white" if vConfusionMatrix[i, j]>thresh else "black")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')

# Grafica matriz de confusión sin normalización y normalizada
def plotConfusionMatrix(sNombreArchivo:str, vConfusionMatrix, lClases):
  fig = plt.figure(figsize=(10, 5))
  #plt.suptitle('Confusion matrix - Station:'+ESTACION+' Channel:'+CANAL)
  plt.subplot(1,2,1)
  plotConfusionMatrixSimple(vConfusionMatrix, lClases, sTitle='Confusion matrix, without normalization')
  plt.subplot(1,2,2)
  plotConfusionMatrixSimple(vConfusionMatrix, lClases, bNormalized=True, sTitle='Normalized confusion matrix')
  fig.tight_layout(pad=2.0)
  #plt.savefig(sNombreArchivo, dpi=100, aspect='normal', bbox_inches='tight', pad_inches=0)
  plt.savefig(sNombreArchivo, dpi=100, bbox_inches='tight', pad_inches=0)
  plt.clf()
  plt.close('all')


# Guardar y recuperar modelos de disco =========================================
# Guardar modelo y atributos
def modelo_guardar(sNombreArchivo:str, tModelo, tFlag, vLossTrain, vLossTest, vAccuracyTrain, vAccuracyTest, tTimeTrain):
    torch.save({
                'carpeta'       :tFlag.carpeta,
                'clases'        :tFlag.classes,
                'num_classes'   :tFlag.num_classes,
                'batch_size'    :tFlag.batch_size,
                'num_workers'   :tFlag.num_workers,
                'learning_rate' :tFlag.learning_rate,
                'num_epochs'    :len(vLossTrain),         # tFlag.num_epochs,
                'time_train'    :tTimeTrain,              # Tiempo de Entrenamiento
                'modelo'        :tModelo.state_dict(),
                #'optimizador'  :optimizer.state_dict(),
                'train_loss'    :vLossTrain,
                'test_loss'     :vLossTest,
                'train_acc'     :vAccuracyTrain,
                'test_acc'      :vAccuracyTest,
                }, sNombreArchivo)

# Carga modelo desde archivo
def modelo_carga(sNombreArchivo:str, tModelo, tFlag):
  # Recuperando valores(modelos) de disco
  tArchivo  = torch.load(sNombreArchivo)

  # Recuperando parametros
  tFlag.carpeta = tArchivo['carpeta']
  tFlag.classes = tArchivo['clases']
  tFlag.num_classes = tArchivo['num_classes']
  tFlag.batch_size  = tArchivo['batch_size']
  tFlag.num_workers = tArchivo['num_workers']
  tFlag.learning_rate = tArchivo['learning_rate']
  tFlag.num_epochs  = tArchivo['num_epochs']
  tTimeTrain        = tArchivo['time_train'] # Tiempo de Entrenamiento
  #fTiempProceso ???
  vLossTrain        = tArchivo['train_loss']
  vLossTest         = tArchivo['test_loss']
  vAccuracyTrain    = tArchivo['train_acc']
  vAccuracyTest     = tArchivo['test_acc']

  # Recuperando modelo
  tModelo.load_state_dict(tArchivo['modelo'])

  # Retornando
  return tModelo, tFlag, vLossTrain, vLossTest, vAccuracyTrain, vAccuracyTest, tTimeTrain


# Métodos para PRUEBA ==========================================================
# Entrenamiento con solo prueba del modelo
def entrenamiento_solo_prueba(tModelo, lossFunction, testLoader, tTiempoEntrenado:int=0):
  # Entrenamiento de modelo que toma eventos reconstruidos
  tTiempoUsado=time.time()

  fTestLoss, fTestAcc   = test_model(tModelo, lossFunction, testLoader)
  print('Test Loss: {:.4f}; Accuracy: {:.4f};  Time:{}  Tiempo Usado:{}s'.format(fTestLoss, fTestAcc, time.asctime(), round(time.time()-tTiempoUsado,1) ))

  tTiempoUsado=(time.time() - tTiempoUsado)/60.0
  # Actualizando tiempo
  tTiempoUsado+=tTiempoEntrenado
  print('Finalizó Prueba de datos nuevos sobre Modelo')
  # Retorno
  return tTiempoUsado, fTestLoss, fTestAcc


## Proceso varios datasets

In [ ]:
for i in range(FLAGS.iPorcentajeTiempoInicio, FLAGS.iPorcentajeTiempoFinal+1, FLAGS.iPorcentajeTiempoInc): # range(FLAGS.iPorcentajeTiempoInicio, FLAGS.iPorcentajeTiempoFinal+1):
  print("Para dataset: ", i, '-', i+FLAGS.iPorcentajeTiempoLong, time.asctime(), '='*70)

  # Abrir dados ================================================================
  #Elimina folder de espectrogramas si existe
  !if [ -d "spectrograms224_00" ]; then rm -rf spectrograms224_00; fi

  # Descomprimir espectrogramas
  !unzip {'"/content/gdrive/MyDrive/Espectrogramas/Escenario2/00/spectrograms224_00 +daRotacion3/spectrograms224_00.'+str(i).zfill(2)+'-'+str(i+FLAGS.iPorcentajeTiempoLong).zfill(2)+'.zip"'} &> /dev/null

  # Crear los dataloaders
  _, trainloader, testloader, iTotal, iTrain, iTest = creaDataLoaders(FLAGS.carpeta, FLAGS.batch_size, FLAGS.num_workers, FLAGS.classes)

  # Entrenar con modelo TL =====================================================
  for sTL in FLAGS.tls:
    print('Inicio Entrenamiento de Modelo:'+sTL, time.asctime())

    # Crea modelo
    TL_Modelo = creaModelo(sTL)

    #GPU =======================================================================
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    #Mover la entrada y modelo TL a GPU para velocidad si esta disponible
    TL_Modelo.to(device)

    #Funcion de pérdida y optimizador ==========================================
    #Función de Pérdida: Entropía cruzada
    loss_function = nn.CrossEntropyLoss()

    #Optimizador: Algoritmo de Adam
    optimizer = optim.Adam(TL_Modelo.parameters(), lr=FLAGS.learning_rate)

    # Entrenamiento y prueba ===================================================
    tTiempoEntrenamiento=time.time()
    vTrainLoss, vTestLoss, vTrainAcc, vTestAcc=[], [], [], []
    for epoch in range(FLAGS.num_epochs):
      train_loss, train_acc = train_model(TL_Modelo, loss_function, optimizer, trainloader)
      test_loss, test_acc = test_model(TL_Modelo, loss_function, testloader)
      vTrainLoss.append(train_loss)
      vTestLoss.append(test_loss)
      vTrainAcc.append(train_acc)
      vTestAcc.append(test_acc)
      if (epoch+1)%5==0:
        print('Época {}/{} Train Loss: {:.4f}; Accuracy: {:.4f} Test Loss: {:.4f}; Accuracy: {:.4f};  Time:{}'.format(epoch + 1, FLAGS.num_epochs, train_loss, train_acc, test_loss, test_acc, time.asctime()))
    tTiempoEntrenamiento=(time.time() - tTiempoEntrenamiento)/60.0
    print('Finalizó Entrenamiento de Modelo:'+sTL, time.asctime())

    # Graficos de Loss y Accuracy ==============================================
    plotLossAcurracy(sTL+'_LossAcc_'+str(i).zfill(2)+'-'+str(i+FLAGS.iPorcentajeTiempoLong).zfill(2)+'.png', range(1, FLAGS.num_epochs+1), vTrainLoss, vTestLoss, vTrainAcc, vTestAcc)

    # Guardando CVS de métricas y calculando matriz de confusión ===============
    vMatrizConfusion = csvMetricas(sTL+'_Metricas_'+str(i).zfill(2)+'-'+str(i+FLAGS.iPorcentajeTiempoLong).zfill(2)+'.csv', TL_Modelo, testloader, FLAGS.classes, FLAGS.num_epochs, iTotal, iTrain, iTest)

    # Gráfico de matriz de confusión
    plotConfusionMatrix(sTL+'_MatConf_'+str(i).zfill(2)+'-'+str(i+FLAGS.iPorcentajeTiempoLong).zfill(2)+'.png', vMatrizConfusion, FLAGS.classes)

    # Mover resultados a Google Drive
    !mv {sTL+'_LossAcc_'+str(i).zfill(2)+'-'+str(i+FLAGS.iPorcentajeTiempoLong).zfill(2)+'.png'}  {'"/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 +daRotacion3 metricas/"'}
    !mv {sTL+'_Metricas_'+str(i).zfill(2)+'-'+str(i+FLAGS.iPorcentajeTiempoLong).zfill(2)+'.csv'} {'"/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 +daRotacion3 metricas/"'}
    !mv {sTL+'_MatConf_'+str(i).zfill(2)+'-'+str(i+FLAGS.iPorcentajeTiempoLong).zfill(2)+'.png'}  {'"/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 +daRotacion3 metricas/"'}

  # Eliminar carpeta de espectrogramas
  !rm -rf spectrograms224_00

  # Destruyendo variables
  del TL_Modelo
  del trainloader
  del testloader
print("FIN DE PROCESO =========")

## (Opcional) Proceso simple dataset

### Descomprimir simple dataset

In [49]:
# Descomprimir dataset
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 210708.zip' &> /dev/null
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 210708 20Hz.zip' &> /dev/null
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 221012 undersampling/spectrograms224_00 221012 10Hz.zip' &> /dev/null
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 +daRotacion4/spectrograms224_00.05-25_20Hz.zip' &> /dev/null
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 +daRotacion4/spectrograms224_00.05-25_20Hz.zip 67095' &> /dev/null
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 +daRotacion3/spectrograms224_00.05-25_20Hz.zip' &> /dev/null
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 210708 20Hz+da_jittering 0.01-0.1.zip' &> /dev/null
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 210708 20Hz+da_jittering 0.001-0.01.zip' &> /dev/null
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 210708 20Hz+da_jittering 0.0001-0.001.zip' &> /dev/null
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 210708 20Hz+da_jittering 0.01-0.1 67095.zip' &> /dev/null
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 210708 20Hz+da_jittering 0.001-0.01 67095.zip' &> /dev/null
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 210708 20Hz+da_jittering 0.0001-0.001 67095.zip' &> /dev/null
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/01/spectrograms224_00 231211 20Hz+da_sa 0.15-2 0.15-2 sin corte ruido.zip' &> /dev/null
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/01/spectrograms224_00 231218 20Hz+da_interpolation 40-60.zip' &> /dev/null
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/01/spectrograms224_00 210708 20Hz+da_jittering 0.2 real+artificial.zip' &> /dev/null
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/01/spectrograms224_00 210708 20Hz+da_jittering 0.2 artificial.zip' &> /dev/null
#!unzip '/content/gdrive/MyDrive/Espectrogramas/Escenario2/01/spectrograms224_00 231211 20Hz+da_ag 5_3.zip' &> /dev/null
#!unzip '/content/gdrive/MyDrive/Espectrogramas/Escenario2/01/spectrograms224_00 231211 20Hz+da_ag 5_5.zip' &> /dev/null
#!unzip '/content/gdrive/MyDrive/Espectrogramas/Escenario2/01/spectrograms224_00 231211 20Hz+da_ag1 0.4_10T.zip' &> /dev/null
#!unzip '/content/gdrive/MyDrive/Espectrogramas/Escenario2/01/spectrograms224_00 231211 20Hz+da_ag1 0.45_10T.zip' &> /dev/null
#!unzip '/content/gdrive/MyDrive/Espectrogramas/Escenario2/01/spectrograms224_00 231211 20Hz+da_sa 0.1-2 0.1-2.zip' &> /dev/null
#!unzip '/content/gdrive/MyDrive/Espectrogramas/Escenario2/01/spectrograms224_00 231211 20Hz+da_sa 0.15-2 0.15-2.zip' &> /dev/null
!unzip '/content/gdrive/MyDrive/Espectrogramas/Escenario2/01/spectrograms224_00 231218 20Hz+da_interpolation 40-60.zip' &> /dev/null

### Extrae muestra del dataset actual de archivos (Opcional)

In [ ]:
import os, glob, shutil
from random import randint, choice, sample

def generaMuestraArchivos(sRutaEntrada:str, sRutaSalida:str, dEvento:dict):
  """Genera una muestra aleatoria de los eventos sin repeticion en carpeta destino.
  Args:
      sRutaEntrada (str): Carpeta donde se encuentan los eventos originales organizado por carpetas Evento
      sRutaSalida (str):  Carpeta donde se generaran la muestra de eventos
      dEvento (dict): Diccionario de eventos y cantidades a generer por evento, Ej. {'HY':2000, 'LP':1500, 'TC':0, 'TR':1800, 'VT':1350}
  """
  # Leyendo eventos
  for sEvento in dEvento:
    print("Generando muestra de eventos:", sEvento)
    # Leyendo lista de archivo de eventos desde carpeta
    #m=fym.lista_archivos_simple(sRutaEntrada+sEvento)
    m=list(map(os.path.basename, glob.glob(sRutaEntrada+sEvento+"/*.*")))
    if len(m)>0:
      # Generando lista que tiene muestra de eventos sin repeticion
      lMuestra = sample(m, dEvento[sEvento])
      # Crear carpetas de salida si no existen
      if not os.path.exists(sRutaSalida+sEvento):
        os.makedirs(sRutaSalida+sEvento)
      # Copiar archivos a ruta de salida
      for fArchivo in lMuestra:
        shutil.copy(sRutaEntrada+sEvento+'/'+fArchivo, sRutaSalida+sEvento)
    # Mensaje
    print("Fin de generación de eventos :", sEvento)
  return

# Crear carpeta temporal
!mkdir spectrograms224
# Muestra a extraer
generaMuestraArchivos('spectrograms224_00/', 'spectrograms224/', {'HY':2686, 'LP':2686, 'TC':2686 ,'TR':2686, 'VT':2686})
#Eliminar carpeta original
!rm -rf spectrograms224_00
# Renombra carpeta
!mv spectrograms224 spectrograms224_00

Generando muestra de eventos: HY
Fin de generación de eventos : HY
Generando muestra de eventos: LP
Fin de generación de eventos : LP
Generando muestra de eventos: TC
Fin de generación de eventos : TC
Generando muestra de eventos: TR
Fin de generación de eventos : TR
Generando muestra de eventos: VT
Fin de generación de eventos : VT


### Entrenar modelo

In [50]:
RUTA_SALIDA=r'/content/gdrive/My Drive/Espectrogramas/Escenario2/01/spectrograms224_00 231218 interpolation/'

print("Para dataset: ", time.asctime(), '='*70)

# Crear los dataloaders
_, trainloader, testloader, iTotal, iTrain, iTest = creaDataLoaders(FLAGS.carpeta, FLAGS.batch_size, FLAGS.num_workers, FLAGS.classes)

# Entrenar con modelo TL =====================================================
for sTL in FLAGS.tls:
  print('Inicio Entrenamiento de Modelo:'+sTL, time.asctime())

  # Crea modelo
  TL_Modelo = creaModelo(sTL)

  #GPU =======================================================================
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  #Mover la entrada y modelo TL a GPU para velocidad si esta disponible
  TL_Modelo.to(device)

  #Funcion de pérdida y optimizador ==========================================
  #Función de Pérdida: Entropía cruzada
  loss_function = nn.CrossEntropyLoss()

  #Optimizador: Algoritmo de Adam
  optimizer = optim.Adam(TL_Modelo.parameters(), lr=FLAGS.learning_rate)

  # Entrenamiento y prueba ===================================================
  tTiempoEntrenamiento=time.time()
  vTrainLoss, vTestLoss, vTrainAcc, vTestAcc=[], [], [], []
  for epoch in range(FLAGS.num_epochs):
    train_loss, train_acc = train_model(TL_Modelo, loss_function, optimizer, trainloader)
    test_loss, test_acc = test_model(TL_Modelo, loss_function, testloader)
    vTrainLoss.append(train_loss)
    vTestLoss.append(test_loss)
    vTrainAcc.append(train_acc)
    vTestAcc.append(test_acc)
    if (epoch+1)%5==0:
      print('Época {}/{} Train Loss: {:.4f}; Accuracy: {:.4f} Test Loss: {:.4f}; Accuracy: {:.4f};  Time:{}'.format(epoch + 1, FLAGS.num_epochs, train_loss, train_acc, test_loss, test_acc, time.asctime()))
  tTiempoEntrenamiento=(time.time() - tTiempoEntrenamiento)/60.0
  print('Finalizó Entrenamiento de Modelo:'+sTL, time.asctime())

  # Graficos de Loss y Accuracy ==============================================
  plotLossAcurracy(RUTA_SALIDA+sTL+'_LossAcc.png', range(1, FLAGS.num_epochs+1), vTrainLoss, vTestLoss, vTrainAcc, vTestAcc)

  # Guardando CVS de métricas y calculando matriz de confusión ===============
  vMatrizConfusion = csvMetricas(RUTA_SALIDA+sTL+'_Metricas.csv', TL_Modelo, testloader, FLAGS.classes, FLAGS.num_epochs, iTotal, iTrain, iTest)

  # Gráfico de matriz de confusión
  plotConfusionMatrix(RUTA_SALIDA+sTL+'_MatConf.png', vMatrizConfusion, FLAGS.classes)

  # Guardar Modelo
  modelo_guardar(RUTA_SALIDA+sTL+'_Modelo.pt', TL_Modelo,  FLAGS, vTrainLoss, vTestLoss, vTrainAcc, vTestAcc, tTiempoEntrenamiento)

  # Mover resultados a Google Drive
  #!mv {sTL+'_LossAcc.png'}  {'"/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 +daRotacion3/"'}
  #!mv {sTL+'_Metricas.csv'} {'"/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 +daRotacion3/"'}
  #!mv {sTL+'_MatConf.png'}  {'"/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 +daRotacion3/"'}
  #!mv {sTL+'_Modelo.pt'}    {'"/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 +daRotacion3/"'}

print("FIN DE PROCESO =========")

Para dataset:  Mon Jun 16 20:39:16 2025 ======================================================================
Clases: ['HY', 'LP', 'TC', 'TR', 'VT'] Datos de Entrenamiento:10744 Prueba:2686 Total:13430
Inicio Entrenamiento de Modelo:AlexNet Mon Jun 16 20:39:16 2025


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Época 5/20 Train Loss: 0.1891; Accuracy: 0.9271 Test Loss: 0.3398; Accuracy: 0.8820;  Time:Mon Jun 16 20:43:25 2025
Época 10/20 Train Loss: 0.0747; Accuracy: 0.9747 Test Loss: 0.4985; Accuracy: 0.8861;  Time:Mon Jun 16 20:47:34 2025
Época 15/20 Train Loss: 0.0500; Accuracy: 0.9814 Test Loss: 0.5064; Accuracy: 0.8831;  Time:Mon Jun 16 20:51:42 2025
Época 20/20 Train Loss: 0.0390; Accuracy: 0.9873 Test Loss: 0.7971; Accuracy: 0.8727;  Time:Mon Jun 16 20:55:51 2025
Finalizó Entrenamiento de Modelo:AlexNet Mon Jun 16 20:55:51 2025
FIN DE PROCESO =========


In [ ]:
# GUARDAR MODELO
#modelo_guardar(RUTA_EXPERIMENTO+MODELO_AE+'ModeloREC.pt', modelNN,  FLAGS_NN, vTrainLoss, vTestLoss, vTrainAcc, vTestAcc, tTiempoEntrenamiento)
#modelo_guardar(RUTA_EXPERIMENTO+MODELO_AE+'ModeloCOD.pt', modelRN,  FLAGS_CC, vTrainLoss, vTestLoss, vTrainAcc, vTestAcc, tTiempoEntrenamiento)

# RECUPERAR MODELO
TL_Modelo, FLAGS, vTrainLoss, vTestLoss, vTrainAcc, vTestAcc, tTiempoEntrenamiento = modelo_carga(
  "/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 +daRotacion3 metricas/AlexNet_Modelo.pt", TL_Modelo, FLAGS)
TL_Modelo.eval()

##(Opcional) Prueba del modelo con datos

### Descomprimir simple dataset

In [55]:
#Elimina folder si es necesario
!rm -rf spectrograms224_00

In [52]:
# Descomprimir dataset
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 210708.zip' &> /dev/null
!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/00/spectrograms224_00 210708 20Hz.zip' &> /dev/null
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/01/spectrograms224_00 231127 20Hz+da_ag1 0.4_10T con corte ruido.zip' &> /dev/null
#!unzip '/content/gdrive/My Drive/Espectrogramas/Escenario2/01/spectrograms224_00 231127 20Hz+da_sa 0.1-2 0.1-2 con corte ruido.zip' &> /dev/null

#### Cargar modelo si no se tiene (OPCIONAL)

In [ ]:
# Constantes
#RUTA_ENTRADA='/content/gdrive/MyDrive/Espectrogramas/Escenario2/00/spectrograms224_00 221012 undersampling/20Hz 20epocas/'
#RUTA_SALIDA ='/content/gdrive/MyDrive/Espectrogramas/Escenario2/00/spectrograms224_00 221012 undersampling/'
#RUTA_ENTRADA='/content/gdrive/MyDrive/Espectrogramas/Escenario2/00/spectrograms224_00 +daJittering 20Hz/0.0001-0.001 67095/'
#RUTA_SALIDA ='/content/gdrive/MyDrive/Espectrogramas/Escenario2/00/spectrograms224_00 +daJittering 20Hz/'
#RUTA_ENTRADA='/content/gdrive/MyDrive/Espectrogramas/Escenario2/00/spectrograms224_00 +daJittering 20Hz/0.0001-0.001 13430 Artificial/'
#RUTA_SALIDA ='/content/gdrive/MyDrive/Espectrogramas/Escenario2/00/spectrograms224_00 +daJittering 20Hz/'
RUTA_ENTRADA='/content/gdrive/MyDrive/Espectrogramas/Escenario2/00/spectrograms224_00 +daJittering 20Hz/0.01-0.1 67095 Artificial/'
RUTA_SALIDA ='/content/gdrive/MyDrive/Espectrogramas/Escenario2/01/spectrograms224_00 231127 sa/'

In [ ]:
# CARGAR MODELO SI NO SE TIENE (OPCIONAL)
# Crea modelo en blanco del requerido
TL_Modelo = creaModelo(TL.AlexNet)

#GPU =======================================================================
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#Mover el modelo TL a GPU para velocidad si esta disponible
TL_Modelo.to(device)

#Función de Pérdida: Entropía cruzada
loss_function = nn.CrossEntropyLoss()

# RECUPERAR MODELO SI NO SE TIENE CARGADO(OCIONAL)
TL_Modelo, FLAGS, vTrainLoss, vTestLoss, vTrainAcc, vTestAcc, tTiempoEntrenamiento = modelo_carga(RUTA_ENTRADA+'AlexNet_Modelo.pt', TL_Modelo, FLAGS)
TL_Modelo.eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

#### Probar el modelo

In [53]:
# Crear los dataloaders
all_dataloader, trainloader, testloader, iTotal, iTrain, iTest = creaDataLoaders(FLAGS.carpeta, FLAGS.batch_size, FLAGS.num_workers, FLAGS.classes)

Clases: ['HY', 'LP', 'TC', 'TR', 'VT'] Datos de Entrenamiento:4916 Prueba:1229 Total:6145


In [54]:
# Se debe tener el modelo TL_Modelo cargado

# Prueba sobre datos de prueba
tTiempoEntrenamiento, vTestLoss, vTestAcc=entrenamiento_solo_prueba(TL_Modelo, loss_function, all_dataloader)

# Métricas
vMatrizConfusionRecPruebaTL = csvMetricas(RUTA_SALIDA+'Rec Real Metricas.csv', TL_Modelo, all_dataloader, FLAGS.classes, -1, iTotal, iTotal, iTotal)
# Gráfico de matriz de confusión
plotConfusionMatrix(RUTA_SALIDA+'Rec Real MatConf.png', vMatrizConfusionRecPruebaTL, FLAGS.classes)

Test Loss: 0.2998; Accuracy: 0.9473;  Time:Mon Jun 16 20:57:12 2025  Tiempo Usado:16.0s
Finalizó Prueba de datos nuevos sobre Modelo
